In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
import scienceplots

sys.path.append('../..')

from libraries import file_management as fm

In [62]:
plt.style.use(['science', 'grid', 'no-latex'])

In [63]:
plot_dict = {
    "energy": "Energy",
    "cos_zenith": "Cosine Zenith",
}


In [2]:
smearing_level = "antares"

In [ ]:
if __name__ == "__main__":
    # Define the path to the ROOT file
    
    path = "/sps/km3net/users/mchadoli/Swim/Data/events/ANTARES"
    root_file = f"antares_w_nnfit_FINAL1_smeared_{smearing_level}.root"
    COLUMNS = [
        "energy_true",
        "cos_zenith_true",
        "energy_smeared",
        "cos_zenith_smeared", 
        ]

    # Load the ROOT file into a DataFrame
    df = fm.load_large_rootfile_to_df(os.path.join(path, root_file), columns=COLUMNS)

    # Rounding the energy and cos_zenith columns
    df["energy_true"] = df["energy_true"].round(2)
    df["cos_zenith_true"] = df["cos_zenith_true"].round(2)

In [66]:
def _plot_hist_fit(
        df,
        parameter = "energy",
        bins=100,
        value= 20,
        smearing_level = "50",
):
    
    
    print(f"Plotting histogram and Gaussian fit for {parameter} with true value {value}")
    smeared_column = f"{parameter}_smeared"
    true_column = f"{parameter}_true"
    data = df[df[true_column] == value]

    # 2. Fit a Gaussian distribution to the data
    mu, std = norm.fit(data[smeared_column])

    # 3. Plot the histogram with counts (density=False automatically uses counts)
    fig, ax = plt.subplots(figsize=(10, 6))
    data.hist(
        column=smeared_column, 
        bins= bins, 
        figsize=(10, 6), 
        ax=ax, 
        histtype='step', 
        hatch = '///',
        color='b', 
        lw=2, 
        label='Histogram')
    
    # 4. Create x values for the fitted Gaussian curve
    xmin, xmax = plt.xlim()  # Get x-axis limits from the histogram
    x_values = np.linspace(xmin, xmax, 1000)  # Create 1000 x values between xmin and xmax

    # 5. Plot the Gaussian curve directly, scaling to match histogram's y-axis (counts)
    p = norm.pdf(x_values, mu, std) * len(data) * (xmax - xmin) / bins

    ax.plot(
        x_values, 
        p, 
        'r--', 
        lw=2, 
        label=f'Fitted Gaussian\n $\\mu$={mu:.2f}, $\\sigma$={std:.2f}')
    
    # 6. Customize plot
    ax.set_xlabel(f'Smeared {plot_dict[parameter]} [GeV]')
    ax.set_ylabel('Counts')
    ax.set_title(f'Histogram and Fitted Gaussian for {plot_dict[parameter]} with true value {value} with smearing level {smearing_level} %')
    ax.legend(frameon=False)

    # 7. Show the plot
    fig.tight_layout()
    fig.savefig(f"../plots/test/histogram_{parameter}_{value}.png")


In [ ]:
for value in [0.5, 0.01, -0.5]:
    _plot_hist_fit(df, parameter="cos_zenith", value=value)